In [608]:
import sklearn
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [609]:
# загружаю данные из файлов
train = pd.read_csv('titanic/train.csv')
test = pd.read_csv('titanic/test.csv')

In [610]:
# просмотр данных для обучения
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [611]:
# просмотр данных для тестов
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [612]:
# создаю вектор результатов
y = train['Survived']
# создаю матрицу фичей для обучения и для тестов
X_train = train[['PassengerId']+['Pclass']+['Sex']+['Age']+['SibSp']+['Parch']+['Fare']] 
X_test = test[['PassengerId']+['Pclass']+['Sex']+['Age']+['SibSp']+['Parch']+['Fare']]

# добавляю фичу - наличие родственников на корабле и удаляю количество Parch, Sibsp
X_train['Relatives'] = (X_train.SibSp + X_train.Parch)
X_test['Relatives'] = (X_test.SibSp + X_test.Parch)
X_train.drop(['SibSp', 'Parch'], axis=1, inplace=True)
X_test.drop(['SibSp', 'Parch'], axis=1, inplace=True)




C:\Users\Slava-N\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Slava-N\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\Slava-N\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
 

In [613]:
# перевожу фичи в qualitative
X_train_qualit = pd.get_dummies(X_train)
X_test_qualit = pd.get_dummies(X_test)


In [614]:
X_train_qualit.head(3)

,PassengerId,Pclass,Age,Fare,Relatives,Sex_female,Sex_male
0,1,3,22.0,7.2500,1,0,1
1,2,1,38.0,71.2833,1,1,0
2,3,3,26.0,7.9250,0,1,0


In [615]:
X_test_qualit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 7 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Age            332 non-null float64
Fare           417 non-null float64
Relatives      418 non-null int64
Sex_female     418 non-null uint8
Sex_male       418 non-null uint8
dtypes: float64(2), int64(3), uint8(2)
memory usage: 17.2 KB


In [616]:
X_train_qualit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
PassengerId    891 non-null int64
Pclass         891 non-null int64
Age            714 non-null float64
Fare           891 non-null float64
Relatives      891 non-null int64
Sex_female     891 non-null uint8
Sex_male       891 non-null uint8
dtypes: float64(2), int64(3), uint8(2)
memory usage: 36.6 KB


In [617]:
# заполняю пропуски с помощью инструмента Imputer
from sklearn.preprocessing import Imputer
imp = Imputer(copy=False)
imp.fit(X_train_qualit)
X_train_qualit_imp = imp.transform(X_train_qualit)
X_test_qualit_imp = imp.transform(X_test_qualit)

In [618]:
# нормализую тестовые и обучающие данные по стандартным параметрам
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train_qualit_imp)
X_train_qualit_imp_scaled = scaler.transform(X_train_qualit_imp)
X_test_qualit_imp_scaled = scaler.transform(X_test_qualit_imp)

In [619]:
# создаю модель логистической регрессии с Lasso regularization
# (потестировал l1 - по ощущениям хуже получалось) и обучаю ее на данных
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(penalty='l2', C=0.1)
model.fit(X_train_qualit_imp_scaled, y)

LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [620]:
# видим, что PassengerId логично имеет коэффициент 0.
print(model.coef_)

[[ 0.0282126  -0.77517583 -0.42037643  0.17662097 -0.33163352  0.62060847
  -0.62060847]]


In [621]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import make_scorer
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
# задаю параметры для кросс-валидации
kfold=KFold(10, True, 2)

In [622]:
# y_predicted = model.predict(X_test_qualit_imp_scaled)

In [623]:
# готовлю пайплайн и gridsearch для поиска наиболее оптимального гиперпараметра C
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

pipeline = Pipeline([('scale', StandardScaler()),
             ('model', LogisticRegression(penalty='l2', C=0.1))])

param_grid={'model__C':[0.0001,0.05,0.1, 0.8, 2.5, 10]}

grid = GridSearchCV(pipeline, param_grid, scoring = make_scorer(accuracy_score), cv=kfold)

In [624]:
from pprint import pprint
model = grid.fit(X_train_qualit_imp,y)
pprint(model.cv_results_)

{'mean_fit_time': array([ 0.00230219,  0.00159981,  0.00180323,  0.00189636,  0.00200016,
        0.00200179]),
 'mean_score_time': array([  8.00728798e-04,   8.96382332e-04,   4.98032570e-04,
         9.96351242e-05,   4.00161743e-04,   3.99732590e-04]),
 'mean_test_score': array([ 0.78338945,  0.7979798 ,  0.79461279,  0.78787879,  0.78900112,
        0.78900112]),
 'mean_train_score': array([ 0.78226749,  0.79847977,  0.800849  ,  0.80035056,  0.80035056,
        0.80035056]),
 'param_model__C': masked_array(data = [0.0001 0.05 0.1 0.8 2.5 10],
             mask = [False False False False False False],
       fill_value = ?)
,
 'params': ({'model__C': 0.0001},
            {'model__C': 0.05},
            {'model__C': 0.1},
            {'model__C': 0.8},
            {'model__C': 2.5},
            {'model__C': 10}),
 'rank_test_score': array([6, 1, 2, 5, 3, 3]),
 'split0_test_score': array([ 0.75555556,  0.73333333,  0.73333333,  0.71111111,  0.71111111,
        0.71111111]),
 'split0_

In [625]:
# генерация прогноза
y_predicted = grid.best_estimator_.predict(X_test_qualit_imp)


In [626]:
best_C = model.best_params_['model__C']

print(best_C)

0.05


In [627]:
# Оценка 
scores = cross_val_score(LogisticRegression(penalty='l2', C=best_C),
                         X_train_qualit_imp_scaled, y, groups=None,
                        scoring = make_scorer(accuracy_score),
                        cv=kfold)
print(scores)

[ 0.73333333  0.7752809   0.83146067  0.87640449  0.78651685  0.7752809
  0.82022472  0.83146067  0.75280899  0.80898876]


In [628]:
# подготовка данных для загрузки в Kaggle. Если есть опция оптимальнее готовить - ПОЖАЛУЙСТА, подскажите.
y_predicted
y_predicted_pd = pd.DataFrame(y_predicted, columns = ['Survived'])
y_predicted_pd.head()


,Survived
0,0
1,0
2,0
3,0
4,1


In [629]:
y_true = pd.read_csv('titanic/true_y.csv')
y_true.head()


,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [630]:
results = pd.DataFrame(list(zip(y_true['PassengerId'], y_predicted_pd['Survived'])), columns=['PassengerId', 'Survived'])

In [632]:
results.to_csv('results_17.csv', index=False, sep=',')